In [ ]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import re
import requests
import time

pd.set_option('display.max_columns', None)

In [ ]:
ump_scores_df = pd.read_csv('D:\Downloads\games (1).csv')

##drop nulls, shortened season
ump_scores_df = ump_scores_df.dropna(axis='index').sort_values('Date')
ump_scores_df = ump_scores_df.set_axis(['date', 'umpire', 'home', 'away', 'home_score', 'away_score', 'called_pitches', 'called_wrong', 'accuracy', 'consistency', 'favor', 'total_run_impact'], axis=1)
covid_2020 = ump_scores_df.date.str.contains('2020')
ump_scores_df = ump_scores_df[~covid_2020].reset_index(drop=True)

##clean for scraping
ump_scores_df.date = ump_scores_df.date.str.replace('-', '')
ump_scores_df.loc[ump_scores_df.home == 'WSH', 'home'] = 'WAS'
ump_scores_df.loc[ump_scores_df.home == 'LAD', 'home'] = 'LAN'
ump_scores_df.loc[ump_scores_df.home == 'SD', 'home'] = 'SDN'
ump_scores_df.loc[ump_scores_df.home == 'SF', 'home'] = 'SFN'
ump_scores_df.loc[ump_scores_df.home == 'NYY', 'home'] = 'NYA'
ump_scores_df.loc[ump_scores_df.home == 'KC', 'home'] = 'KCA'
ump_scores_df.loc[ump_scores_df.home == 'STL', 'home'] = 'SLN'
ump_scores_df.loc[ump_scores_df.home == 'NYM', 'home'] = 'NYN'
ump_scores_df.loc[ump_scores_df.home == 'TB', 'home'] = 'TBA'
ump_scores_df.loc[ump_scores_df.home == 'CWS', 'home'] = 'CHA'
ump_scores_df.loc[ump_scores_df.home == 'CHC', 'home'] = 'CHN'
display(ump_scores_df)

ump_scores_df.groupby('date')['home_score']

In [ ]:
##webscrape to pull start_time, attendance, duration, day_night
url = 'https://www.baseball-reference.com/boxes/'

scorebox = []
start_time = []
duration = []
day_night = []
for i, j in zip(ump_scores_df.date, ump_scores_df.home):
    response = requests.get(url + j + '/' + j + i + '0.shtml')
    soup = bs(response.text, 'html.parser')
    print(j)
    print(i)
    print(response)
    scorebox = soup.find_all('div', {'class':'scorebox_meta'})
    start_time.append(scorebox)
    duration.append(scorebox)
    day_night.append(scorebox)
    time.sleep(3)

##estimated time to run: ~11 hours
##started: 10:34 AM EDT
##ended: 11:12 PM EDT

In [ ]:
##save lists to dataframe then export as csv to be used elsewhere
start_time_df = pd.DataFrame(start_time)
duration_df = pd.DataFrame(duration)
day_night_df = pd.DataFrame(day_night)
start_time_df.to_csv('start_time_df.csv')
duration_df.to_csv('duration_df.csv')
day_night_df.to_csv('day_night_df.csv')